In [1]:
#install python 3.9 and dev utils
#you may not need all the dev libraries, but I haven't tested which aren't necessary.
!sudo apt-get update -y
!sudo apt-get install python3.9 python3.9-dev python3.9-distutils libpython3.9-dev

#change alternatives
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 1
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 2

# install pip
!curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py
!python3 get-pip.py --force-reinstall

#install colab's dependencies
!python3 -m pip install ipython ipython_genutils ipykernel jupyter_console prompt_toolkit httplib2 astor

# link to the old google package
!ln -s /usr/local/lib/python3.7/dist-packages/google \
       /usr/local/lib/python3.9/dist-packages/google



Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [1,039 kB]
Get:1

In [3]:
! pip install datasets transformers rouge-score nltk
!apt install git-lfs
!pip install sentencepiece
!pip install pandas==1.4.1
!pip install torchvision 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached torchvision-0.14.0-cp39-cp39-manylinux1_x86_64.whl (24.3 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 890.2/890.2 MB 62.0 MB/s eta 0:00:01tcmalloc: large alloc 1112711168 bytes == 0x38e72000 @  0x7f8fb64a0615 0x60596b 0x543df1 0x543f6b 0x5440

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


4.25.1


In [1]:
import transformers

import nltk
nltk.download('punkt')
import numpy as np

print(transformers.__version__)

from transformers import AutoTokenizer
import torch
from datasets import load_metric
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

/usr/local/lib/python3.9/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


4.25.1


In [2]:
import pandas as pd
print(pd.__version__)

1.4.1


In [3]:
#df = pd.read_pickle("/content/drive/MyDrive/WikiFlow/enwiki-FullArticles.pkl")
df = pd.read_pickle("/content/drive/MyDrive/WikiFlow/enwiki-latest-pages-articles1-p1p41242.pkl")
df['truncated_text'] = df['text'].apply(lambda x: " ".join(x[0:800]))
df

,title,id,text,truncated_text
0,Anarchism,12,"[anarchism, is, political, philosophy, and, mo...",anarchism is political philosophy and movement...
1,Albedo,39,"[diffusely, reflected, sunlight, relative, to,...",diffusely reflected sunlight relative to vario...
2,A,290,"[or, is, the, first, letter, and, the, first, ...",or is the first letter and the first vowel of ...
3,Alabama,303,"[alabama, is, state, in, the, southeastern, re...",alabama is state in the southeastern region of...
4,Achilles,305,"[pottery, painting, dating, to, bc, of, achill...",pottery painting dating to bc of achilles duri...
...,...,...,...,...
20693,Heuristic routing,41236,"[heuristic, routing, is, system, used, to, des...",heuristic routing is system used to describe h...
20694,Hierarchical routing,41237,"[hierarchical, routing, is, method, of, routin...",hierarchical routing is method of routing in n...
20695,High-performance equipment,41239,"[high, performance, equipment, describes, equi...",high performance equipment describes equipment...
20696,Hop,41240,"[hop, is, type, of, jump, hop, or, hops, may, ...",hop is type of jump hop or hops may also refer...


In [4]:
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/WikiFlow/Test3")
model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/WikiFlow/Test3")
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [7]:
i = 0
print(df['title'][i])
ex = df['truncated_text'][i]
print(ex)

Anarchism
anarchism is political philosophy and movement that is skeptical of all justifications for authority and seeks to abolish the institutions they claim maintain unnecessary coercion and hierarchy typically including though not necessarily limited to governments nation states and capitalism anarchism advocates for the replacement of the state with stateless societies or other forms of free associations as historically left wing movement usually placed on the farthest left of the political spectrum it is usually described alongside communalism and libertarian marxism as the libertarian wing libertarian socialism of the socialist movement humans lived in societies without formal hierarchies long before the establishment of formal states realms or empires with the rise of organised hierarchical bodies scepticism toward authority also rose although traces of anarchist thought are found throughout history modern anarchism emerged from the enlightenment during the latter half of the t

In [9]:
summarize(ex)

Token indices sequence length is longer than the specified maximum sequence length for this model (1424 > 512). Running this sequence through the model will result in indexing errors


"Anarchism is often used as a symbol of libertarian anti-government movement in the Middle Ages, and fascinates revolutionaries to be radicalised by modern capitalism but has become increasingly popular with liberal groups like Marxian or Islamist people. But it seems that these movements have been delineated for centuries before becoming more widely disdrew on social media than those who are not ideologically divided from ideologic leaders within their societies which were formed during this period including civil war; they've seen an archetypic rhetorical ideas suggest suggesting there was no disparity between theories among scholars"

In [5]:
def summarize(x):
  torch.cuda.empty_cache()
  tokenized_text = tokenizer.encode(x, return_tensors="pt").to(device)
  summary_ids = model.generate(tokenized_text,
                                      num_beams=4,
                                      no_repeat_ngram_size=1,
                                      min_length=32,
                                      max_length=64,
                                      early_stopping=True)
  output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
  return output
  #print(output)

In [6]:
summaries = []
from tqdm import tqdm
for t in tqdm(df['truncated_text']):
  
  summaries.append(summarize(t))


100%|██████████| 20698/20698 [4:11:26<00:00,  1.37it/s]


In [10]:
df['summary'] = summaries
df

,title,id,text,truncated_text,summary
0,Anarchism,12,"[anarchism, is, political, philosophy, and, mo...",anarchism is political philosophy and movement...,Anarchist movement of libertarian socialism ha...
1,Albedo,39,"[diffusely, reflected, sunlight, relative, to,...",diffusely reflected sunlight relative to vario...,Earth surface albedo is the directional reflec...
2,A,290,"[or, is, the, first, letter, and, the, first, ...",or is the first letter and the first vowel of ...,'Ox head boeotian greek letter written by the ...
3,Alabama,303,"[alabama, is, state, in, the, southeastern, re...",alabama is state in the southeastern region of...,Alabama capital is based on automotive finance...
4,Achilles,305,"[pottery, painting, dating, to, bc, of, achill...",pottery painting dating to bc of achilles duri...,achilles was written by statius in the 18th ce...
...,...,...,...,...,...
20693,Heuristic routing,41236,"[heuristic, routing, is, system, used, to, des...",heuristic routing is system used to describe h...,The heuristic routing system is used to determ...
20694,Hierarchical routing,41237,"[hierarchical, routing, is, method, of, routin...",hierarchical routing is method of routing in n...,The routers are based on hierarchical routing ...
20695,High-performance equipment,41239,"[high, performance, equipment, describes, equi...",high performance equipment describes equipment...,The performance characteristics required for u...
20696,Hop,41240,"[hop, is, type, of, jump, hop, or, hops, may, ...",hop is type of jump hop or hops may also refer...,Texas steel guitar player places hop river con...


In [7]:
import pickle
with open('/content/drive/MyDrive/WikiFlow/summaries.pickle', 'wb') as handle:
    pickle.dump(summaries, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [11]:
df.to_pickle("/content/drive/MyDrive/WikiFlow/Wiki_summaries.pkl")  

tcmalloc: large alloc 1073741824 bytes == 0xe0b6c000 @  0x7f31bfecd1e7 0x5bf416 0x53b894 0x538378 0x538864 0x538864 0x4759cc 0x53c52f 0x5385d3 0x537de7 0x53c424 0x5385d3 0x475973 0x537de7 0x53c250 0x5385d3 0x5397d1 0x53c52f 0x5385d3 0x53a12d 0x645413 0x55c171 0x4ae642 0x5bcb62 0x5f4604 0x4ae642 0x5bcb62 0x508ca6 0x4ade91 0x5bcb62 0x6924fb
tcmalloc: large alloc 2147483648 bytes == 0x1213b6000 @  0x7f31bfecd1e7 0x5bf416 0x53b894 0x538378 0x538864 0x538864 0x4759cc 0x53c52f 0x5385d3 0x537de7 0x53c424 0x5385d3 0x475973 0x537de7 0x53c250 0x5385d3 0x5397d1 0x53c52f 0x5385d3 0x53a12d 0x645413 0x55c171 0x4ae642 0x5bcb62 0x5f4604 0x4ae642 0x5bcb62 0x508ca6 0x4ade91 0x5bcb62 0x6924fb


In [15]:
summaries[120]

'Aquarius is one of the oldest known constellations on Earth today. It belongs to an Egyptian goddess who sacrificed water rather than wine, and it was also found in ancient Greek mythology by its famous fishie-like creatures that have been created for humans since 1939; they are now called aqua'